In [201]:
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

In [211]:
#ds = pd.read_csv('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
url= 'https://raw.githubusercontent.com/nandex7/Capstone-Project---Azure-Machine-Learning-Engineer/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
ds = TabularDatasetFactory.from_delimited_files(path=url,validate='False',separator=',',infer_column_types=True,include_path=False,
set_column_types=None,support_multi_line=False,partition_format=None)

#ds.head(5)


In [212]:
df = ds.to_pandas_dataframe()
df.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [213]:
data =ds

In [214]:
 x_df = data.to_pandas_dataframe().dropna()

In [215]:
x_df.Partner.unique()

array([ True, False])

In [216]:
print(df.columns)

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


In [217]:
x_df.drop('customerID',inplace=True, axis=1)

In [218]:

x_df['Partner']= x_df.Partner.apply(lambda s:1 if s==True else 0)
x_df['gender'] =x_df.gender.apply(lambda s:1 if s=="Male" else 0)
x_df['Dependents']= x_df.Dependents.apply(lambda s:1 if s==True else 0)
x_df['PhoneService']=x_df.PhoneService.apply(lambda s:1 if s==True else 0)
x_df['PaperlessBilling']=x_df.PaperlessBilling.apply(lambda s:1 if s==True else 0)
x_df['Churn']=x_df.Churn.apply(lambda s:1 if s==True else 0)


x_df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,1,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,1,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,1,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,0,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,1,Mailed check,84.80,1990.50,0
7039,0,0,1,1,72,1,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,1,Credit card (automatic),103.20,7362.90,0
7040,0,0,1,1,11,0,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,1,Electronic check,29.60,346.45,0
7041,1,1,1,0,4,1,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Mailed check,74.40,306.60,1


In [219]:
    categorical= ['MultipleLines','InternetService'
    , 'OnlineBackup','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV', 'StreamingMovies', 'Contract','PaymentMethod']

In [220]:
 x_df = pd.get_dummies(x_df, columns = categorical)

In [224]:
x_df.Churn.unique()

array([0, 1], dtype=int64)

In [225]:
#x_df.drop(columns=categorical, inplace=True, axis=1)
x_df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,0,...,1,0,0,1,0,0,0,0,1,0
1,1,0,0,0,34,1,0,56.95,1889.50,0,...,1,0,0,0,1,0,0,0,0,1
2,1,0,0,0,2,1,1,53.85,108.15,1,...,1,0,0,1,0,0,0,0,0,1
3,1,0,0,0,45,0,0,42.30,1840.75,0,...,1,0,0,0,1,0,1,0,0,0
4,0,0,0,0,2,1,1,70.70,151.65,1,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,1,84.80,1990.50,0,...,0,0,1,0,1,0,0,0,0,1
7039,0,0,1,1,72,1,1,103.20,7362.90,0,...,0,0,1,0,1,0,0,1,0,0
7040,0,0,1,1,11,0,1,29.60,346.45,0,...,1,0,0,1,0,0,0,0,1,0
7041,1,1,1,0,4,1,1,74.40,306.60,1,...,1,0,0,1,0,0,0,0,0,1


In [223]:
x_df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges',
       'Churn', 'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credi

In [179]:
def clean_data(data):
    # Dict for cleaning data
    
    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    #x_df = data.dropna()
    #Boolean Values
    x_df.drop('customerID',inplace=True, axis=1)
    x_df['Partner']= x_df.Partner.apply(lambda s:1 if s==True else 0)
    x_df['gender'] =x_df.gender.apply(lambda s:1 if s=="Male" else 0)
    x_df['Dependents']= x_df.Dependents.apply(lambda s:1 if s==True else 0)
    x_df['PhoneService']=x_df.PhoneService.apply(lambda s:1 if s==True else 0)
    x_df['PaperlessBilling']=x_df.PaperlessBilling.apply(lambda s:1 if s==True else 0)
    x_df['Churn']=x_df.Churn.apply(lambda s:1 if s==True else 0)

    
    #Categorical Values Hot Encoding.
    categorical= ['MultipleLines','InternetService'
    , 'OnlineBackup','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV', 'StreamingMovies', 'Contract','PaymentMethod']
    x_df = pd.get_dummies(x_df, columns = categorical)
    
    y_df = x_df.Churn
    x_df = x_df.drop('Churn', axis=1)
    return x_df, y_df
    

In [180]:
x_df.Churn.unique()

array([0], dtype=int64)

In [181]:
x, y = clean_data(ds)

#x.head(5)
#y.head(5)
# TODO: Split data into train and test sets.
### YOUR CODE HERE ###a
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.33, random_state=42)

run = Run.get_context()


In [182]:
print(x_train.size)
print(x_test.size)

188440
92840


In [137]:
x_df.size

288312

In [183]:
x_df.Churn.unique()

array([0], dtype=int64)

In [ ]:
def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

    args = parser.parse_args()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))

    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))

    os.makedirs('outputs', exist_ok=True)
    joblib.dump(model, 'outputs/model.joblib')

if __name__ == '__main__':
    main()